Import cleaned data to start the EDA

In [370]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
import matplotlib.pyplot as plt
import rasterio
import geopandas as geopd
import rasterio.rio
import seaborn as sns
import datetime as dt 

from rasterio.plot import show

import pyreadr



In [371]:
foxes_all = geopd.read_file("../data/cleaned_shapefiles/foxes_all.shp")
foxes_resamp = geopd.read_file("../data/cleaned_shapefiles/foxes_resamp.shp")
sample_points = geopd.read_file("../data/cleaned_shapefiles/sample_points.shp")

In [372]:
foxes_all.id.value_counts()

#2019-FSBD609-002

2019-FSBD609-002          8337
2019-FSBD624-004          7862
2019-FSBD641_y-gr/r-b     6899
2019-FSBD197-005          5351
2018-FSBD641_b-r/gr-b     4231
2018-FSBD619_r-gr/r-y     4111
2018-FSBD615_v-gr/r-gr    4075
2019-FSAC008-012          3124
2019-FSBD622-006          2248
2019-FSAC008-011          1693
2019-FSBD615-001          1372
2018-FSBD608-001           830
Name: id, dtype: int64

In [373]:
from datetime import datetime, timedelta
from shapely.geometry import Point
import geopandas as gpd
import math

import geopy.distance


def get_time_diffs(df, df_time_col = "t_"):
    fmt = '%Y-%m-%d-%H:%M:%S'
    time_diff = []
    travel_distance = []
    for i, fox in enumerate(df.t_):
        if i > 0 and df.iloc[i-1].id == df.iloc[i].id :
            time_1 = datetime.strptime(df.iloc[i-1][df_time_col], fmt)
            time_2 = datetime.strptime(df.iloc[i][df_time_col], fmt)
            time_diff.append((time_2 - time_1).total_seconds())
        else:
            time_diff.append(np.nan)
    return time_diff
    
def get_distance(df): 
  travel_distance = []
  for i, fox in enumerate(df.t_):
    if i > 0 and df.id.iloc[i-1] == df.id.iloc[i]:
      x_diff = df.x_.iloc[i-1] - df.x_.iloc[i]
      y_diff = df.y_.iloc[i-1] - df.y_.iloc[i]
      tot_diff = np.sqrt(np.square(x_diff) + np.square(y_diff) )
      travel_distance.append(tot_diff)
    else:
      travel_distance.append(np.nan)
  return travel_distance

foxes_all["travel_distance"] = get_distance(foxes_all)
foxes_all["time_diff"] = get_time_diffs(foxes_all)
 
foxes_all.eval('travel_speed = (travel_distance / 1000 ) / (time_diff / 3600)', inplace = True)  




In [374]:
foxes_all["fox_day"] = [str(datetime.strptime(x, '%Y-%m-%d-%H:%M:%S' ) + timedelta(hours=12))[:10]  for x in foxes_all.t_ ]
foxes_all

,x_,y_,t_,id,sex,NDVI,NDMI,soil,veg,slope,aspect,elev,timestamp,geometry,travel_distance,time_diff,travel_speed,fox_day
0,548522.717454,7.353167e+06,2018-07-10-01:56:30,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531181e+09,POINT (548522.717 7353166.639),NaN,NaN,NaN,2018-07-10
1,548525.680513,7.353168e+06,2018-07-10-06:56:32,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531199e+09,POINT (548525.681 7353168.363),3.427867,18002.0,0.000685,2018-07-10
2,548539.963810,7.353164e+06,2018-07-10-11:56:32,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531217e+09,POINT (548539.964 7353164.150),14.891535,18000.0,0.002978,2018-07-10
3,548195.507163,7.352493e+06,2018-07-10-22:31:11,2018-FSBD608-001,F,0.443307,-0.138425,Stone,Grassland,10.065748,339.629059,1072.429321,1.531255e+09,POINT (548195.507 7352493.197),754.207680,38079.0,0.071303,2018-07-11
4,548558.891944,7.352589e+06,2018-07-11-13:31:01,2018-FSBD608-001,F,0.504280,-0.113833,Stone,Grassland,12.232175,6.898972,1056.955200,1.531309e+09,POINT (548558.892 7352588.648),375.711810,53990.0,0.025052,2018-07-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50128,543632.466003,7.379183e+06,2019-12-14-08:10:19,2019-FSBD641_y-gr/r-b,M,0.386280,-0.137583,Stone,Stone,6.499063,325.394470,1100.971313,1.576307e+09,POINT (543632.466 7379183.144),9449.328321,14405.0,2.361512,2019-12-14
50129,544607.109028,7.378558e+06,2019-12-14-12:10:15,2019-FSBD641_y-gr/r-b,M,0.248821,-0.214249,Roesberg,Dry Shrub,11.062690,351.290924,1213.619629,1.576322e+09,POINT (544607.109 7378558.328),1157.723289,14396.0,0.289511,2019-12-15
50130,544147.656802,7.379631e+06,2019-12-14-16:10:13,2019-FSBD641_y-gr/r-b,M,0.000000,-1.000000,Stone,Dry Shrub,14.816979,351.992798,1012.299194,1.576336e+09,POINT (544147.657 7379630.549),1166.513787,14398.0,0.291669,2019-12-15
50131,539474.780875,7.379241e+06,2019-12-15-00:10:14,2019-FSBD641_y-gr/r-b,M,0.000000,-1.000000,Stone,Stone,18.056763,350.892731,1169.776489,1.576365e+09,POINT (539474.781 7379241.118),4689.075177,28801.0,0.586114,2019-12-15


In [375]:
d = foxes_all.geometry
gdf = gpd.GeoDataFrame(d, crs=3006)


gdf = gdf.to_crs(epsg= 4326)

foxes_all["geo_kepler_lat"] = [geo.y for geo in gdf.geometry]
foxes_all["geo_kepler_lon"] = [geo.x for geo in gdf.geometry]

foxes_all


,x_,y_,t_,id,sex,NDVI,NDMI,soil,veg,slope,aspect,elev,timestamp,geometry,travel_distance,time_diff,travel_speed,fox_day,geo_kepler_lat,geo_kepler_lon
0,548522.717454,7.353167e+06,2018-07-10-01:56:30,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531181e+09,POINT (548522.717 7353166.639),NaN,NaN,NaN,2018-07-10,66.294597,16.081633
1,548525.680513,7.353168e+06,2018-07-10-06:56:32,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531199e+09,POINT (548525.681 7353168.363),3.427867,18002.0,0.000685,2018-07-10,66.294612,16.081700
2,548539.963810,7.353164e+06,2018-07-10-11:56:32,2018-FSBD608-001,F,0.662722,0.125608,Moraine,Dry Shrub,10.135784,205.492111,1033.016602,1.531217e+09,POINT (548539.964 7353164.150),14.891535,18000.0,0.002978,2018-07-10,66.294572,16.082017
3,548195.507163,7.352493e+06,2018-07-10-22:31:11,2018-FSBD608-001,F,0.443307,-0.138425,Stone,Grassland,10.065748,339.629059,1072.429321,1.531255e+09,POINT (548195.507 7352493.197),754.207680,38079.0,0.071303,2018-07-11,66.288607,16.074083
4,548558.891944,7.352589e+06,2018-07-11-13:31:01,2018-FSBD608-001,F,0.504280,-0.113833,Stone,Grassland,12.232175,6.898972,1056.955200,1.531309e+09,POINT (548558.892 7352588.648),375.711810,53990.0,0.025052,2018-07-12,66.289407,16.082217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50128,543632.466003,7.379183e+06,2019-12-14-08:10:19,2019-FSBD641_y-gr/r-b,M,0.386280,-0.137583,Stone,Stone,6.499063,325.394470,1100.971313,1.576307e+09,POINT (543632.466 7379183.144),9449.328321,14405.0,2.361512,2019-12-14,66.528680,15.981750
50129,544607.109028,7.378558e+06,2019-12-14-12:10:15,2019-FSBD641_y-gr/r-b,M,0.248821,-0.214249,Roesberg,Dry Shrub,11.062690,351.290924,1213.619629,1.576322e+09,POINT (544607.109 7378558.328),1157.723289,14396.0,0.289511,2019-12-15,66.522937,16.003450
50130,544147.656802,7.379631e+06,2019-12-14-16:10:13,2019-FSBD641_y-gr/r-b,M,0.000000,-1.000000,Stone,Dry Shrub,14.816979,351.992798,1012.299194,1.576336e+09,POINT (544147.657 7379630.549),1166.513787,14398.0,0.291669,2019-12-15,66.532620,15.993500
50131,539474.780875,7.379241e+06,2019-12-15-00:10:14,2019-FSBD641_y-gr/r-b,M,0.000000,-1.000000,Stone,Stone,18.056763,350.892731,1169.776489,1.576365e+09,POINT (539474.781 7379241.118),4689.075177,28801.0,0.586114,2019-12-15,66.529758,15.888233


In [376]:
def df_to_geojson_trip(df, properties, lat='geo_kepler_lat', lon='geo_kepler_lon', elev = 'elev', time = 'timestamp'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'LineString',
                               'coordinates':[]}}
    for i, row in df.iterrows():
        
        feature['geometry']['coordinates'].append([row[lon], row[lat], row[elev], row[time]])
    for prop in properties:
        feature['properties'][prop] = row[prop]
    geojson['features'].append(feature)
    return geojson

In [383]:
cols = ['id']#, 'timestamp']#ä, 'geo_kepler_lat', 'geo_kepler_lon']#, 'geometry']
geojson = df_to_geojson_trip(foxes_all.query('id == "2019-FSBD609-002"'), cols)
geojson = df_to_geojson_trip(foxes_all, cols)
#geojson

In [384]:
import json

output_filename = '../data/out/foxes_for_kepler.geojson'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False, default=vars)

In [385]:
foxes_all.id.unique()

array(['2018-FSBD608-001', '2018-FSBD615_v-gr/r-gr',
       '2018-FSBD619_r-gr/r-y', '2018-FSBD641_b-r/gr-b',
       '2019-FSAC008-011', '2019-FSAC008-012', '2019-FSBD197-005',
       '2019-FSBD609-002', '2019-FSBD615-001', '2019-FSBD622-006',
       '2019-FSBD624-004', '2019-FSBD641_y-gr/r-b'], dtype=object)

In [386]:
points_per_day = foxes_all[["id", "time_diff", "fox_day"]].groupby(["id","fox_day"], as_index=False ).count().rename(columns={"time_diff": "points_this_day"})

max_window_per_day = foxes_all[["id", "time_diff", "fox_day"]].groupby(["id","fox_day"], as_index=False ).max().rename(columns={"time_diff": "max_window"})

a = pd.merge( foxes_all,points_per_day, left_on=["id", "fox_day"], right_on=["id", "fox_day"] )
b = pd.merge( max_window_per_day, a , left_on=["id", "fox_day"], right_on=["id", "fox_day"] )
b[["id", "fox_day", "t_", "time_diff", "points_this_day", "max_window"]]

b.query("max_window < 7300 and points_this_day > 10").groupby(["id", "fox_day"], as_index=False ).count().max() 

id                 2019-FSBD641_y-gr/r-b
fox_day                       2020-12-17
max_window                            96
x_                                    96
y_                                    96
t_                                    96
sex                                   96
NDVI                                  96
NDMI                                  96
soil                                  96
veg                                   96
slope                                 96
aspect                                96
elev                                  96
timestamp                             96
geometry                              96
travel_distance                       96
time_diff                             96
travel_speed                          96
geo_kepler_lat                        96
geo_kepler_lon                        96
points_this_day                       96
dtype: object